I need Chemical Formula, Cell Parameters, Space Group

In [1]:
import plotly.graph_objects as go
import numpy as np

def plotting_struct(structure):
    x, y, z = structure[:,0] , structure[:,1], structure[:,2]
    fig = go.Figure(data=[go.Scatter3d(
        x = x,
        y = y,
        z = z,
        mode='markers',
        marker=dict(
            size=5,
            opacity=0.8
        )
    )])
    
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig.show()

In [33]:
from pymatgen.io.cif import CifParser
def make_super_cell(cif_path, num_atoms, cut):
    # Create a CIF parser and parse the CIF file
    parser = CifParser(cif_path)

    structure = parser.parse_structures(primitive= False, symmetrized=True)[0]
    
    for site in structure:
        # Round fractional coordinates for each atom
        site.frac_coords = [round(coord, 4) for coord in site.frac_coords]
        
    # Print information about the crystal structure
    print("Lattice Parameters:")
    print("a =", structure.lattice.a)
    print("b =", structure.lattice.b)
    print("c =", structure.lattice.c)
    print("alpha =", structure.lattice.alpha)
    print("beta =", structure.lattice.beta)
    print("gamma =", structure.lattice.gamma)
    print()

    print("Space Group:", structure.get_space_group_info())
    print()

    print("Asymmetric Unit:")
    for site in structure.sites:
        print(f"{site.specie}: {site.frac_coords}")
        
    unit_cell = structure.make_supercell(num_atoms)

    unit_cell_coordinates = unit_cell.cart_coords
    
    if cut == True:
        limit = structure.lattice.a - (structure.lattice.a/num_atoms) + 0.0001

        conditions = (unit_cell_coordinates[:,0] <= limit) & (unit_cell_coordinates[:,1] <= limit) & (unit_cell_coordinates[:,2] <= limit)
        unit_cell_coordinates[~conditions] = np.nan

    plotting_struct(unit_cell_coordinates)

In [34]:
make_super_cell("CsCl.cif", 2, True)

Lattice Parameters:
a = 4.14369834
b = 4.14369834
c = 4.14369834
alpha = 90.0
beta = 90.0
gamma = 90.0

Space Group: ('Pm-3m', 221)

Asymmetric Unit:
Cl-: [0.5 0.5 0.5]
Cs+: [0. 0. 0.]


In [35]:
make_super_cell("CsCl.cif", 2, False)

Lattice Parameters:
a = 4.14369834
b = 4.14369834
c = 4.14369834
alpha = 90.0
beta = 90.0
gamma = 90.0

Space Group: ('Pm-3m', 221)

Asymmetric Unit:
Cl-: [0.5 0.5 0.5]
Cs+: [0. 0. 0.]


In [23]:
make_super_cell("C.cif", 2, False)

Lattice Parameters:
a = 2.46729126
b = 2.46729126
c = 7.803073
alpha = 90.0
beta = 90.0
gamma = 120.00000000000001

Space Group: ('P6_3/mmc', 194)

Asymmetric Unit:
C: [0.   0.   0.25]
C: [0.   0.   0.75]
C: [0.3333 0.6667 0.25  ]
C: [0.6667 0.3333 0.75  ]


In [30]:
gamma = 120
a = 2
b = a


a_dist = 2.46729126
b_dist = 2.46729126

print((a_dist) + (np.cos(np.deg2rad(180 - gamma)) * b_dist))

3.7009368900000004
